In [ ]:

!pip install numpy opencv-python


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab import files
from ipywidgets import interact, IntSlider

class SeamCarver:
    def __init__(self, image):
        self.original = image.copy()
        self.reset()
        self.removed_seams_count = 0
        self.removed_seams_history = []  # Track all removed seams

    def reset(self):
        """Reset to original image"""
        self.current = self.original.copy()
        self.height, self.width = self.current.shape[:2]
        self.gray = cv2.cvtColor(self.current, cv2.COLOR_BGR2GRAY).astype(np.float32)
        self.energy = self._compute_energy()
        self.removed_seams_count = 0
        self.removed_seams_history = []

    def _compute_energy(self, update_cols=None):
      """Compute energy incrementally for affected columns only"""
      h, w = self.height, self.width
      dx = np.zeros_like(self.gray)
      dy = np.zeros_like(self.gray)

      if update_cols is None:
          # Full computation (initial step)
          dx[:, 1:-1] = np.abs(self.gray[:, 2:] - self.gray[:, :-2])
          dx[:, 0] = np.abs(self.gray[:, 1] - self.gray[:, 0])
          dx[:, -1] = np.abs(self.gray[:, -1] - self.gray[:, -2])
          dy[1:-1, :] = np.abs(self.gray[2:, :] - self.gray[:-2, :])
          dy[0, :] = np.abs(self.gray[1, :] - self.gray[0, :])
          dy[-1, :] = np.abs(self.gray[-1, :] - self.gray[-2, :])
      else:
          # Partial update for affected columns
          cols = np.unique(np.clip(update_cols, 0, w-1))
          for c in cols:
              # Update horizontal gradient (dx) for columns ±2 around c
              left = max(c-2, 0)
              right = min(c+3, w)
              dx[:, left:right-1] = np.abs(self.gray[:, left+1:right] - self.gray[:, left:right-1])

              # Update vertical gradient (dy) for the entire column c
              if h > 2:
                  dy[1:-1, c] = np.abs(self.gray[2:, c] - self.gray[:-2, c])
              dy[0, c] = np.abs(self.gray[1, c] - self.gray[0, c])
              dy[-1, c] = np.abs(self.gray[-1, c] - self.gray[-2, c])

      return dx + dy

    def _compute_cumulative_energy(self):
        """Bottom-up DP (Section 3)"""
        cum_energy = self.energy.copy()
        for i in range(1, self.height):
            left = np.roll(cum_energy[i-1], 1)
            left[0] = np.inf
            right = np.roll(cum_energy[i-1], -1)
            right[-1] = np.inf
            cum_energy[i] += np.minimum(np.minimum(left, cum_energy[i-1]), right)
        return cum_energy

    def _find_vertical_seam(self):
        cum_energy = self._compute_cumulative_energy()
        seam = np.zeros(self.height, dtype=int)
        seam[-1] = np.argmin(cum_energy[-1])
        for i in range(self.height-2, -1, -1):
            j_prev = seam[i+1]
            j_min = max(j_prev-1, 0)
            j_max = min(j_prev+1, self.width-1)
            seam[i] = j_min + np.argmin(cum_energy[i, j_min:j_max+1])
        return seam

    def _remove_vertical_seam(self, seam):
        mask = np.ones((self.height, self.width), dtype=bool)
        mask[np.arange(self.height), seam] = False
        self.current = self.current[mask].reshape((self.height, self.width-1, 3))
        self.gray = self.gray[mask].reshape((self.height, self.width-1))
        self.energy = self.energy[mask].reshape((self.height, self.width-1))
        self.width -= 1
        self.removed_seams_count += 1
        self.removed_seams_history.append(seam.copy())

    def carve_to_width(self, target_width):
        """Carve until reaching target width"""
        if target_width < 1 or target_width > self.original.shape[1]:
            return
        if target_width < self.width:
            while self.width > target_width:
                self._remove_vertical_seam(self._find_vertical_seam())
        elif target_width > self.width:
            self.reset()
            self.carve_to_width(target_width)

    def get_visualization(self):
        """Create visualization with seams and info overlay"""
        viz = self.original.copy()
        for seam in self.removed_seams_history[:self.removed_seams_count]:
            for row, col in enumerate(seam):
                if 0 <= col < viz.shape[1]:
                    viz[row, col] = [0, 0, 255]  # Red color

        # Add info text
        text = f"Seams removed: {self.removed_seams_count}"
        cv2.putText(viz, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                   0.8, (255, 255, 255), 2, cv2.LINE_AA)
        return viz

def upload_image():
    """Upload an image file"""
    uploaded = files.upload()
    for filename in uploaded.keys():
        return cv2.imdecode(np.frombuffer(uploaded[filename], np.uint8), cv2.IMREAD_COLOR)
    return None

def display_results(original, resized, viz):
    """Display original, resized, and visualization side-by-side"""
    plt.figure(figsize=(18, 6))

    # Original Image
    plt.subplot(1, 3, 1)
    plt.title("Original Image")
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    # Resized Image
    plt.subplot(1, 3, 2)
    plt.title("Resized Image")
    plt.imshow(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    # Seam Visualization
    plt.subplot(1, 3, 3)
    plt.title("Seam Visualization")
    plt.imshow(cv2.cvtColor(viz, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.show()

def interactive_carving(target_width):
    """Interactive carving with slider"""
    global carver
    carver.carve_to_width(target_width)
    resized = carver.current
    viz = carver.get_visualization()
    display_results(carver.original, resized, viz)

# Main execution
if __name__ == "__main__":
    # Upload image
    print("Upload an image:")
    img = upload_image()
    if img is None:
        raise ValueError("No image uploaded!")

    # Initialize carver
    carver = SeamCarver(img)
    max_width = img.shape[1]

    # Create interactive slider
    interact(interactive_carving,
             target_width=IntSlider(min=1, max=max_width, step=1, value=max_width))